<a href="https://colab.research.google.com/github/railman-misaka/twikit2/blob/main/twikit_%E3%83%9D%E3%82%B9%E3%83%88%E3%83%87%E3%83%BC%E3%82%BF%E5%8F%96%E5%BE%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# twikitポスト検索
# search_tweet、get_user_tweetsでポストのデータを取得します
※詳細は[公式ドキュメントを見て下さい](https://twikit.readthedocs.io/en/latest/twikit.html#twikit.client.client.Client.search_tweet)<br>
ポストから取得できる項目も[ここで確認して下さい](https://twikit.readthedocs.io/en/latest/twikit.html#twikit.tweet.Tweet)




## twikitインストール

In [ ]:
# 最初にインストールして下さい
!pip install twikit -q

# 認証処理

## アカウント認証

In [ ]:
import asyncio
from twikit import Client

# 日本語なら"ja"
client = Client("ja")

USERNAME = "**************"
PASSWORD = "**************"
# EMAIL = "**************"

# EMAILは不要と思われますがエラーの場合はセットしてみて下さい
await client.login(
    auth_info_1=USERNAME,
    # auth_info_2=EMAIL,
    password=PASSWORD
)

### Cookieを保存して下さい

In [ ]:
client.save_cookies("cookies.json")

# ※2回目以降は保存したCookieで認証して下さい

In [ ]:
import asyncio
from twikit import Client

# 日本語なら"ja"
client = Client("ja")

client.load_cookies("cookies.json")

## Guestの場合（※アカウント不要ですがget_user_tweetsしかできません）

In [ ]:
import asyncio
from twikit.guest import GuestClient

client = GuestClient()

await client.activate()

## **キーワードでポストを取得（search_tweet） ※Guest不可**

## ポストを話題順で取得します

In [ ]:
keyword = "パリオリンピック"

# キーワードからポスト取得（"Top"は話題順）
tweets = await client.search_tweet(query=keyword, product="Top")

print(str(len(tweets)) + "件")

for tweet in tweets:

    # id、日付(UTC)、内容(改行削除)、インプレッション、いいね、リツイート数を出力
    print(tweet.id, tweet.created_at_datetime, tweet.text.replace("\n", " "), tweet.view_count, tweet.favorite_count, tweet.retweet_count)

## ポストを新しい順に取得します
**※queryにsince:～ until:～でポストされた期間を指定できます**

In [ ]:
keyword = "パリオリンピック since:2024-8-1 until:2024-8-3"

# キーワードからポスト取得（"Latest"は最新順）
tweets = await client.search_tweet(query=keyword, product="Latest")

print(str(len(tweets)) + "件")

for tweet in tweets:

    # id、日付(UTC)、内容(改行削除)、インプレッション、いいね、リツイート数を出力
    print(tweet.id, tweet.created_at_datetime, tweet.text.replace("\n", " "), tweet.view_count, tweet.favorite_count, tweet.retweet_count)

## ユーザを指定して新しい順に取得します
**※search_tweetでもfrom:(ユーザ名)でユーザのポストが検索できるようです**

In [ ]:
keyword = "from:sayaka_okada219 since:2024-8-4 until:2024-8-5"

# キーワードからポスト取得（"Latest"は最新順）
tweets = await client.search_tweet(query=keyword, product="Latest")

print(str(len(tweets)) + "件")

for tweet in tweets:

    # id、日付(UTC)、内容(改行削除)、インプレッション、いいね、リツイート数を出力
    print(tweet.id, tweet.created_at_datetime, tweet.text.replace("\n", " "), tweet.view_count, tweet.favorite_count, tweet.retweet_count)

# **ユーザを指定してポストを取得（get_user_tweets）**

## ユーザを指定してポストを新しい順に取得します

In [ ]:
from datetime import datetime

# ユーザ名（@～のやつ）をセット
user_name = "sayaka_okada219"

user = await client.get_user_by_screen_name(user_name)

tweets = await client.get_user_tweets(user.id, "Tweets")

print(str(len(tweets)) + "件")

for tweet in tweets:

    print(tweet.id, tweet.created_at, tweet.text.replace("\n", " "), tweet.view_count, tweet.favorite_count, tweet.retweet_count)

# Dataframe変換

## Guest以外の場合

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

df = pd.DataFrame(columns=["created_at", "text", "view_count", "favorite_count", "retweet_count"])

for tweet in tweets:

        # インデックスをidにして日付(日本時間に変換)、内容、インプレッション、いいね、リツイート数を出力
        df.loc[tweet.id] = [tweet.created_at_datetime.replace(tzinfo=None)+timedelta(hours=9), tweet.text, int(tweet.view_count or 0), int(tweet.favorite_count or 0), int(tweet.retweet_count or 0)]

# インデックス名設定
df.index.name = "tweet_id"

## Guestの場合  ※created_atを変換します

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

df = pd.DataFrame(columns=["created_at", "text", "view_count", "favorite_count", "retweet_count"])

for tweet in tweets:

        # インデックスをidにして日付(日本時間に変換)、内容、インプレッション、いいね、リツイート数を出力
        df.loc[tweet.id] = [datetime.strptime(tweet.created_at,'%a %b %d %H:%M:%S %z %Y').replace(tzinfo=None)+timedelta(hours=9), tweet.text, int(tweet.view_count or 0), int(tweet.favorite_count or 0), int(tweet.retweet_count or 0)]

# インデックス名設定
df.index.name = "tweet_id"

## データの抽出

In [ ]:
# 日付（日本時間で1日前）で抽出
dt = datetime.now() + timedelta(hours=9) - timedelta(days=1)

df_day = df.query('created_at > @dt')

print(df_day)

# インプレッション数で抽出
num = 10000

df_imp = df.query('view_count > @num')

print(df_imp)

## CSVにダウンロード

In [ ]:
file_name = "data.csv"

df.to_csv(file_name)